In [1]:
import pandas as pd
import numpy as np
import os
from  mda_core.time import Time

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [3]:
try:
    DataFilePath_usermotion = f'./user_motion_1.csv'
    DataFilePath_rxdata = f'./Receiver_data.csv'

except:
    raise FileExistsError("File not found")

In [4]:
rxdata = pd.DataFrame()
user = pd.DataFrame()
errorpos = pd.DataFrame()
errorvel = pd.DataFrame()

In [5]:
rxdata = pd.read_csv(DataFilePath_rxdata)
rxdata.columns

Index(['Number of GPS satellites in view and tracking',
       'Number of GAGAN satellites in view and tracking',
       'GPS time of fix Day', 'GPS time of fix Month', 'GPS time of fix Year',
       'GPS time of fix Hour', 'GPS time of fix Minute',
       'GPS time of fix Second', 'GPS TOW', 'Measurement latch time',
       'GPS Week', 'X position', 'Y position', 'Z position', 'X velocity',
       'Y velocity', 'Z velocity', 'PDOP', 'HDOP', 'VDOP', 'GDOP',
       'Number of satellites used for fix', 'Position validity', 'Latitude',
       'Longitude', 'Altitude', 'GPS to UTC offset', 'BIST validities',
       'Ionospheric and Tropospheric corrections applied', 'Clock bias',
       'Clock drift'],
      dtype='object')

In [6]:
user = pd.read_csv(DataFilePath_usermotion)
user.head()

,Time_ms(ms),UposX(m),UposY(m),UposZ(m),UvelX(m/s),UvelY(m/s),UvelZ(m/s),UaccX(m/s2),UaccY(m/s2),UaccZ(m/s2),...,Uheight(m),Uheading(rad),Uele(rad),Ubank(rad),Uang_velX(rad/s),Uang_velY(rad/s),Uang_velZ(rad/s),Uang_accX(rad/s2),Uang_accY(rad/s2),Uang_accZ(rad/s2)
0,58000,5294475.432,4462052.423,450961.9306,593.338374,-1475.181882,7496.132315,-6.512748,-5.394151,-0.540229,...,560595.4316,0.133048,0.0,0.0,0,0,0,0,0,0
1,58100,5294534.733,4461904.878,451711.5411,592.687091,-1475.721284,7496.078248,-6.512899,-5.393882,-0.541128,...,560594.8622,0.133049,0.0,0.0,0,0,0,0,0,0
2,58200,5294593.969,4461757.279,452461.1462,592.035794,-1476.260659,7496.024090,-6.513050,-5.393613,-0.542026,...,560594.2932,0.133050,0.0,0.0,0,0,0,0,0,0
3,58300,5294653.140,4461609.626,453210.7459,591.384481,-1476.800007,7495.969842,-6.513200,-5.393344,-0.542924,...,560593.7246,0.133051,0.0,0.0,0,0,0,0,0,0
4,58400,5294712.246,4461461.919,453960.3402,590.733154,-1477.339328,7495.915505,-6.513351,-5.393075,-0.543822,...,560593.1564,0.133052,0.0,0.0,0,0,0,0,0,0


In [7]:
cols_rxdata = ['Position validity','X position','Y position','Z position','X velocity','Y velocity','Z velocity']
for cols in rxdata.columns:
    if cols not in cols_rxdata:
        rxdata = rxdata.drop(cols,axis=1)
rxdata.head()

,X position,Y position,Z position,X velocity,Y velocity,Z velocity,Position validity
0,5294479.0,4462056.5,451000.9688,593.21,-1474.61,7495.8799,2
1,5295069.0,4460579.0,458496.6875,586.69,-1480.16,7495.4302,2
2,5295652.5,4459096.5,465992.0313,580.38,-1485.85,7495.0498,2
3,5296229.0,4457608.0,473487.2813,573.88,-1491.21,7494.5098,2
4,5296801.5,4456113.5,480979.8750,567.39,-1496.64,7493.9399,2


In [8]:
cols_userdata = ['Time_ms(ms)','UposX(m)','UposY(m)','UposZ(m)','UvelX(m/s)','UvelY(m/s)','UvelZ(m/s)']
for cols in user.columns:
    if cols not in cols_userdata:
        user = user.drop(cols,axis=1)
user = user[user['Time_ms(ms)']%1000 == 0]
user = user.reset_index(drop=True)
user.head()

,Time_ms(ms),UposX(m),UposY(m),UposZ(m),UvelX(m/s),UvelY(m/s),UvelZ(m/s)
0,58000,5294475.432,4462052.423,450961.9306,593.338374,-1475.181882,7496.132315
1,59000,5295065.514,4460574.544,458457.7913,586.824872,-1480.574688,7495.587595
2,60000,5295649.081,4459091.275,465953.1028,580.309867,-1485.964798,7495.033893
3,61000,5296226.133,4457602.616,473447.8561,573.793368,-1491.352205,7494.471210
4,62000,5296796.667,4456108.571,480942.0422,567.275383,-1496.736903,7493.899547


In [12]:
errorpos['Time(s)'] = user['Time_ms(ms)']*0.001
errorvel['Time(s)'] = user['Time_ms(ms)']*0.001
errorpos['Px'] = abs(user['UposX(m)'] - rxdata['X position'])
errorpos['Py'] = abs(user['UposY(m)'] - rxdata['Y position'])
errorpos['Pz'] = abs(user['UposZ(m)'] - rxdata['Z position'])
errorpos['Pos Error norm'] = (errorpos.iloc[:,1:]**2).sum(1).pow(1/2)
errorpos['Position Validity'] = rxdata['Position validity']
errorvel['Vx'] = abs(user['UvelX(m/s)'] - rxdata['X velocity'])
errorvel['Vy'] = abs(user['UvelY(m/s)'] - rxdata['Y velocity'])
errorvel['Vz'] = abs(user['UvelZ(m/s)'] - rxdata['Z velocity'])
errorvel['Vel Error RMS'] = (errorvel.iloc[:,1:]**2).sum(1).pow(1/2)
errorpos.head()

,Time(s),Px,Py,Pz,Pos Error norm,Position Validity
0,58.0,3.568,4.077,39.0382,55.737485,2
1,59.0,3.486,4.456,38.8962,55.586374,2
2,60.0,3.419,5.225,38.9285,55.756942,2
3,61.0,2.867,5.384,39.4252,56.419031,2
4,62.0,4.833,4.929,37.8328,54.387015,2


In [10]:
errorvel.head()
# (df.iloc[:,1:]**2).sum(1).pow(1/2)

,Time(s),Vx,Vy,Vz,Vel Error RMS
0,58.0,0.128374,0.571882,0.252415,0.638155
1,59.0,0.134872,0.414688,0.157395,0.463605
2,60.0,0.070133,0.114798,0.015907,0.135463
3,61.0,0.086632,0.142205,0.038590,0.170929
4,62.0,0.114617,0.096903,0.040353,0.155421


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=errorpos['Time(s)'],y=errorpos['Px']))
fig.add_trace(go.Scatter(x=errorpos['Time(s)'],y=errorpos['Py']))
fig.add_trace(go.Scatter(x=errorpos['Time(s)'],y=errorpos['Pz']))
plot_font = dict(family="Courier New, monospace", size=18, color="RebeccaPurple")
fig.update_layout(
font_family="Courier New",
font_color="black",
title_font_family="Times New Roman",
title_font_color="red",
legend_title_font_color="green",
title = 'Receiver Position Error(m)'
)